In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import math
import random
import time

from collections import defaultdict



In [ ]:
from tf_rl.simulation import DoublePendulum

In [59]:
DOUBLE_PENDULUM_PARAMS = {
    'g_ms2': 9.8, # acceleration due to gravity, in m/s^2
    'l1_m': 1.0, # length of pendulum 1 in m
    'l2_m': 2.0, # length of pendulum 2 in m
    'm1_kg': 1.0, # mass of pendulum 1 in kg
    'm2_kg': 1.0, # mass of pendulum 2 in kg
    'damping': 0.4,
    'max_control_input': 20.0
}

In [60]:
d = DoublePendulum(DOUBLE_PENDULUM_PARAMS)

In [61]:
d.perform_action(0.2)

In [62]:
simulate(d, fps=30, actions_per_simulation_second=1, speed=1.0, simulation_resultion=0.01)

<tf_rl.utils.svg.Scene instance at 0x7ff30ce45248>

KeyboardInterrupt: 

In [133]:
for i in range(100):
    d.step(0.1)
print(d.collect_reward())
d.to_html(['siema = 11.'])

0.221796985246


<tf_rl.utils.svg.Scene instance at 0x7f0241c7a9e0>

0.70284282958419297